In [ ]:
%pip install librosa

In [ ]:
from os import listdir
from os.path import isfile, join

def get_directory_files(dir, with_extenton=False):

  directoy_files = [f for f in listdir(dir) if isfile(join(dir, f))]

  # remove extension
  if not with_extenton:
    directoy_files = [f.split(".")[0] for f in directoy_files]

  return directoy_files

In [ ]:
import re

def note_to_midi(note):
    note_map = {'C': 0, 'C#': 1, 'D': 2, 'D#': 3, 'E': 4, 'F': 5, 'F#': 6, 'G': 7, 'G#': 8, 'A': 9, 'A#': 10, 'B': 11}
    match = re.match(r'([A-G]#?)(\d)v(\d+)', note)
    if match:
        note_name, octave, version = match.groups()
        if version == '1':  # only consider version 1
            midi_number = note_map[note_name] + int(octave) * 12
            return midi_number
    return None  # return None if the note doesn't match the pattern or is not version 1

In [ ]:
import librosa
import soundfile as sf

def transpose_audio_file(filepath, semitones):
    # Load the audio file
    y, sr = librosa.load(filepath, sr=None)
    
    # Transpose it by the desired number of semitones
    y_transposed = librosa.effects.pitch_shift(y=y, sr=sr, n_steps=semitones)
    
    # Save the transposed file
    new_filepath = filepath.replace('.wav', f'_transposed{semitones}.wav')
    sf.write(new_filepath, y_transposed, sr)

    return new_filepath



In [ ]:
import shutil
import os

def create_lib(original_files):
  new_dir = 'samples'
  os.makedirs(new_dir, exist_ok=True)
  
  # Save existing files in new directory with MIDI number as filename
  for file, midi_num in original_files:
      new_filename = os.path.join(new_dir, f'key-{midi_num:03d}.wav')
      shutil.copyfile(file, new_filename)

  # Fill in missing MIDI numbers by transposing previous .wav file
  for midi_num in range(9, 97):  # MIDI numbers from 9 to 96
      filepath = os.path.join(new_dir, f'key-{midi_num:03d}.wav')
      
      # If the .wav file doesn't exist, transpose the previous .wav file
      if not os.path.exists(filepath):
          previous_filepath = os.path.join(new_dir, f'key-{(midi_num-1):03d}.wav')
          new_filepath = transpose_audio_file(previous_filepath, 1)  # Transpose up by 1 semitone
          os.rename(new_filepath, filepath)  # Rename the new file to have the correct MIDI number

In [ ]:
raw_dir = '44.1khz16bit/'
files = get_directory_files(raw_dir, with_extenton=True)
files = [(raw_dir+f, note_to_midi(f)) for f in files]
files = [f for f in files if f[1] is not None]
files = sorted(files, key=lambda x: x[1])
create_lib(files)